In [4]:

import os
os.getcwd() 
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
from test import *
warnings.filterwarnings("ignore")
from datasets import load_dataset,load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch_optimizer as optim
from transformers.optimization import Adafactor, AdafactorSchedule
import torch.backends.cudnn as cudnn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler
from torch.autograd import Variable
import logging
import sys
import transformers
from basic_model import *
import time
import argparse
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import string
from cycle import *
from train import *
from utils import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('t5-small')
def d(l):
    return tokenizer.batch_decode(l,skip_special_tokens=True)
def en(l):
    return tokenize(l,tokenizer,512,True)

In [9]:
en(['.'])
label = en(['Sehr geehrte Kollegen! Ich danke Frau Schroedter für den fundierten Bericht.', 'Sie hat sich eingehend mit der Problematik beschäftigt und während der Behandlung im Ausschuß viele zu dem vorliegenden Bericht eingegangene nderungsanträge berücksichtigt.'])
pred = en(['Herr Kommissar, ich möchte Frau Schroedter für einen ausgezeichneten Bericht danken.', 'Sie hat in der Aussprache im Ausschuss viele der nderungsanträge berücksichtigt, die zu diesem Bericht eingereicht wurden.'])

In [3]:
model = torch.load('./model/D_A.pt').eval()

In [21]:
#real sentences
x = label[0].cuda()
#delte 0 give huge diff
       
# x = torch.Tensor(x).long().cuda()
print(x)
print(x.shape)
x_attn = (x>0.5).long()

print('\t\t---------embedding')
x_emb = model.embedding(x)
print(x_emb.shape)
print(x_emb)# same for padding

print('\t\t---------encode')
distr = model.encoder(inputs_embeds=x_emb,attention_mask=x_attn).last_hidden_state#(bs,sentence length,512)
print(distr.shape)
print(distr)

print('\t\t---------mask the vector')
x_attn_unsq= x_attn.unsqueeze(-1)
distr = torch.mul(distr,x_attn_unsq)#previously
print(distr.shape)
print(distr)


print('\t\t---------mean of the unmask ')
distr = torch.sum(distr,1)/torch.sum(x_attn,1)
print(distr.shape)
print(distr)

print('\t\t---------classifer ')
ret =  model.classifier(distr)#(bs,1)
print(ret)

tensor([[    3, 18976,   873,  7392,    17,    15, 23420,    55,  1674, 12957,
            15,  7672,   180, 10363,    15,    26,   449,   218,   177,  3069,
          6305, 19146,     5,     1,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  292,     3,   547,   289,   236,  8615,    26,   181,    74,  5289,
             9,  4414, 23029,    64,     3,  5729,    74, 12196,   256,  1392,
           860,  6712,  2584,   170,   340,   426, 22646, 19146,   236, 16416,
            15,     3, 20181,     7,   288,  7921,   397, 29699,     5,     1]],
       device='cuda:0')
torch.Size([2, 40])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')
		---------embedding
torc

RuntimeError: The size of tensor a (512) must match the size of tensor b (2) at non-singleton dimension 1

In [29]:
#real sentences
x = pred[0].cuda()
       
# x = torch.Tensor(x).long().cuda()
print(x.shape)
x_attn = (x>0.5).long()

print('\t\t---------embedding')
x_emb = model.embedding(x)
print(x_emb.shape)
print(x_emb)# same for padding

print('\t\t---------encode')
distr = model.encoder(inputs_embeds=x_emb,attention_mask=x_attn).last_hidden_state#(bs,sentence length,512)
print(distr.shape)
print(distr)

print('\t\t---------mask the vector')
x_attn_unsq= x_attn.unsqueeze(-1)
distr = torch.mul(distr,x_attn_unsq)#previously
print(distr.shape)
print(distr)


print('\t\t---------mean of the unmask ')
print(torch.sum(distr,1).shape)
distr = torch.sum(distr,1)/torch.sum(x_attn,1)
print(distr.shape)
print(distr)

print('\t\t---------classifer ')
ret =  model.classifier(distr)#(bs,1)
print(ret)

torch.Size([2, 31])
		---------embedding
torch.Size([2, 31, 512])
tensor([[[-21.8447, -25.4242,  -1.9711,  ...,  14.6676,  42.0918,  13.0946],
         [ 14.2375,  -0.1345,  11.7293,  ...,  31.9208,  14.0575,   8.5395],
         [  5.0407, -21.2402,   6.1542,  ...,  38.0355,  10.9177,  38.7281],
         ...,
         [ -1.9703,   0.1800,  -7.0921,  ...,  -0.4834,   2.7068,  -2.8691],
         [ -1.9703,   0.1800,  -7.0921,  ...,  -0.4834,   2.7068,  -2.8691],
         [ -1.9703,   0.1800,  -7.0921,  ...,  -0.4834,   2.7068,  -2.8691]],

        [[ -9.1765,  -7.9843,  17.2429,  ...,  18.7044,  16.9157,  18.5393],
         [ 11.2275,   8.0182,  14.2276,  ...,   9.7521,  -7.9230,  -3.6829],
         [ 15.2374,  -6.6709, -12.6626,  ...,   3.4055,  -7.8125,  -6.7354],
         ...,
         [ 15.8901, -35.9702, -16.5363,  ..., -29.2687,   1.1392,   2.1299],
         [ -4.3322,   7.3662, -14.5843,  ...,  13.0421,  -2.7197,  -3.1223],
         [ 12.6452,   8.1971, -11.5302,  ...,   7.8170,  

RuntimeError: The size of tensor a (512) must match the size of tensor b (2) at non-singleton dimension 1

In [45]:

# x_emb = model.embedding(x_)#
# print('x_emb.shape',x_emb.shape)
# distr = model.encoder(inputs_embeds=x_emb).last_hidden_state#(bs,sentence length,512)
# print('distr.shape',distr.shape)
# x_attn= x_attn.unsqueeze(-1)
# print('x_attn.shape',x_attn.shape)
# distr = torch.mul(distr,x_attn)#previously

# print('mutil',torch.sum(distr,1))
# print('count',torch.sum(x_attn,1))
# distr = torch.sum(distr,1)/torch.sum(x_attn,1)#(bs,512)
# print(torch.mean(distr,-1))
# print('mea',distr.shape)

# ret =  model.classifier(distr)#(bs,1)
# print(ret)
# ret = model.relu(ret)#(bs,1)
# print(ret)



a_pred_dis = model(x_,x_attn)
a_pred_dis

tensor([[0.9008]], device='cuda:0', grad_fn=<ReluBackward0>)

,attention_mask=x_attn is essential for the encoder inpuit


## We previously use torch.mean to (x after embedding)*x_attn but the mean will be small for the long sentences, now we use torch.sum(distr,1)/torch.sum(x_attn,1)

## We can see that the D judge the sentence by it's length
## small output for the first part, but large for the latter part, so the longer the higher

## so we change the output to [batch,0,512] ie, onlythe first word's sumQ*V and than droppout and then classifier